# 测试包含插件的浏览器会话

本笔记本演示了如何使用PlaywrightSessionPool获取和使用包含插件的浏览器会话。

In [ ]:
headless = False  # 全局设置

In [ ]:
import getpass
import os, sys, asyncio

print("Current user:", getpass.getuser())
print("UID:", os.getuid())
if sys.platform == "win32":
    # 设置 ProactorEventLoop 策略
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

sys.stderr.fileno = lambda: 2
sys.path.append(os.path.join(os.getcwd(), '..'))

In [ ]:
import random
from app.models.RPA_browser.browser_info_model import UserBrowserInfoListParams, UserBrowserInfoCreateParams
from app.utils.depends.session_manager import DatabaseSessionManager
import sys
import asyncio

from app.services.RPA_browser.browser_db_service import BrowserDBService

if sys.platform == 'win32':
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

from app.services.RPA_browser.browser_session_pool.playwright_pool import get_default_session_pool
import uuid

In [ ]:
# 创建或获取浏览器指纹
browser_token = uuid.UUID("0fc69198bccb4b00928b9372c99190b7")

async with DatabaseSessionManager.async_session() as session:
    browser_list = await BrowserDBService.list_fingerprint(UserBrowserInfoListParams(
        browser_token=browser_token,
        page=1,
        per_page=10
    ), session)

if browser_list.items:
    browser_id = browser_list.items[0].id
else:
    async with DatabaseSessionManager.async_session() as session:
        created_fingerprint = await BrowserDBService.create_fingerprint(UserBrowserInfoCreateParams(
            browser_token=browser_token,
            fingerprint_int=random.randint(0, 9999)
        ), session)
        browser_id = created_fingerprint.id

print(f"Using browser_token: {browser_token}")
print(f"Using browser_id: {browser_id}")

In [ ]:
# 使用PlaywrightSessionPool获取包含插件的浏览器会话
from app.models.RPA_browser.browser_session_model import SessionCreateParams

# 获取会话池
session_pool = get_default_session_pool()

# 创建会话参数
session_params = SessionCreateParams(
    browser_token=browser_token,
    browser_id=browser_id,
    headless=headless
)

# 获取包含插件的会话
plugined_session = await session_pool.get_session(session_params)
print(f"Got plugined session for browser_id: {browser_id}")

In [ ]:
page = await plugined_session.get_current_page()

下面可以写需要测试的脚本了，写完了丢到自定义的类里就行

In [ ]:
await page.goto("https://live.bilibili.com/")
await page.click(".header-login-entry")
await page.fill(".bili-mini-account input", 'xxx')
await page.fill('.bili-mini-password input', 'xxx')
await page.click(".universal-btn.login-btn")

In [ ]:
a = await page.locator(".geetest_panel.geetest_wind .geetest_item_wrap:last-of-type").evaluate(r"""el=>{
                                const style = window.getComputedStyle(el);
                                const bg = style.backgroundImage;
                                const match = bg.match(/url\(["']?(.*?)["']?\)/);
                                return match ? match[1] : null;
                                }""")
print(a)
print(page.is_closed())

In [ ]:
from app.services.geetest.captcha_break import acb

geetest_url = await page.locator(".geetest_panel.geetest_wind .geetest_item_wrap:last-of-type").evaluate(r"""el=>{
                                const style = window.getComputedStyle(el);
                                const bg = style.backgroundImage;
                                const match = bg.match(/url\(["']?(.*?)["']?\)/);
                                return match ? match[1] : null;
                                }""")
print(f"geetest_url:{geetest_url}")
result = await acb.predict_chinese_click_from_url(url=geetest_url)

In [ ]:
bound = await page.locator(".geetest_item_wrap:last-of-type").bounding_box()
for x, y in result:
    print(x, y)
    print(bound)
    click_position = {"x": x / 360 * bound.get('width'), "y": y / 360 * bound.get('height')}
    print(click_position)
    await asyncio.sleep(1)
    await page.locator(".geetest_table_box").click(position=click_position)
await page.screenshot(path='./geetest.png')


In [ ]:
await page.locator(".geetest_commit_tip").click()

In [ ]:
await page.screenshot(path='./login.png')

In [ ]:
# await page.goto("https://bot-detector.rebrowser.net/")
# await page.screenshot(path='./1.png',full_page=True)

In [21]:
res = await page.evaluate("()=>{return window.__LIVE_USER_LOGIN_STATUS__ }")

2025-12-11 19:34:49 | INFO        | app.services.site_rpa_operation.plugins.log_plugin:_log_start_operation:50 | [LOG PLUGIN] 🚀 开始执行操作 - 时间戳: 1765452889.73
2025-12-11 19:34:49 | DEBUG       | app.services.site_rpa_operation.plugins.log_plugin:_log_start_operation:51 | [LOG PLUGIN] 📋 配置信息 - 日志级别: INFO, 插件描述: 用户级别的默认日志记录插件
2025-12-11 19:34:49 | DEBUG       | app.services.site_rpa_operation.plugins.log_plugin:_log_operation_context:56 | [LOG PLUGIN] 📋 操作上下文 - 浏览器引擎: BaseUndetectedPlaywright
2025-12-11 19:34:49 | DEBUG       | app.services.site_rpa_operation.plugins.log_plugin:_log_operation_context:57 | [LOG PLUGIN] 📋 操作上下文 - 会话状态: 已连接
2025-12-11 19:34:49 | INFO        | app.services.site_rpa_operation.plugins.page_limit_plugin:_check_page_limit:33 | [PAGE LIMIT PLUGIN] 📊 当前页面数量: 1/5
2025-12-11 19:34:49 | DEBUG       | app.services.site_rpa_operation.plugins.page_limit_plugin:_check_page_limit:34 | [PAGE LIMIT PLUGIN] 📋 页面使用率: 20.0%
2025-12-11 19:34:49 | INFO        | app.services.site_rp

In [22]:
print(res)

{'uid': 0, 'isLogin': False, 'face': '', 'uname': '', 'isError': False}
